In [ ]:
# 1835514122

# EDA

In [2]:
import pandas as pd

In [2]:
music_data = pd.read_excel("Lyrics Test.xlsx")

In [17]:
music_data.columns

Index(['song_id', 'song_id.1', 'lyrics', 'tlyrics'], dtype='object')

In [18]:
music_data[music_data['song_id.1'] == 1835514122]

,song_id,song_id.1,lyrics,tlyrics
183,183,1835514122,[00:00.000] 作词 : Doja Cat/David Sprecher/Rogét...,[00:09.164]\n[00:09.165]我们紧拥整夜 迸发爱火\n[00:12.54...


In [19]:
music_data[music_data['song_id.1'] == 1835514122]['lyrics']

183    [00:00.000] 作词 : Doja Cat/David Sprecher/Rogét...
Name: lyrics, dtype: object

In [4]:
df = music_data.copy()

In [5]:
df = df.dropna(subset='lyrics')

In [26]:
df['lyrics']

0      ในวันที่โลกนั้นเงียบเหงา แค่มีเธอเดินเข้ามา\nอ...
1                               作曲 : Keita Haga\n纯音乐，请欣赏
2      What seems to be endless\nPointless, colorless...
3      编曲 : PDP楽\n好沮丧 the Summer’s Gone 气温又降了 不知不觉又添了...
4      作词 : Seiji Nayuta/ナナヲアカリ\n作曲 : Seiji Nayuta\nあ...
                             ...                        
524    作词 : DLyn\n作曲 : DLyn\n*Huidong Gangsta*\n光光 Ga...
525    作词 : 藤林聖子\n作曲 : 田中公平\n编曲 : 田中公平/村瀬恭久\n深海へ 眠った物...
526    作词 : 桃子\n作曲 : 赵二\nEverything should forget\nev...
527                                              纯音乐，请欣赏
528    We hug and yes, we make love\nAnd always just ...
Name: lyrics, Length: 526, dtype: object

In [34]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\Fatima
[nltk_data]     Azfar\AppData\Roaming\nltk_data...


True

In [32]:
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
import string
from bs4 import BeautifulSoup
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics.pairwise import cosine_similarity 

def clean_string(text):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', ' ', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]
    
    wnl = WordNetLemmatizer()
    
    text_lemmed = [wnl.lemmatize(word) for word in text]

    text_lemmed = text_filtered

    final_string = ' '.join(text_lemmed)

    return final_string

[nltk_data] Downloading package stopwords to C:\Users\Fatima
[nltk_data]     Azfar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
df["lyrics"] = [clean_string(lyrics) for lyrics in df["lyrics"]]

In [39]:
df['lyrics'][183]

'作词 doja catdavid sprecherrogét chahayedcarter langgerard powell iisolana rowe 作曲 doja catdavid sprecherrogét chahayedcarter langgerard powell iisolana rowe doja cat hug yes make love always say goodnight lalalalala cuddle sure love need lips mine doja cat kiss young boy aint got nothin lose oh oh principle baby hold cause like way groove oh oh doja cat boy write name ooh love taste ohlalalalala tongue want boy write name ooh love taste ohlalalalala tongue want doja cat feel like ing something could corny sugar aint dummydummy likе say kiss cut rubbish might bе somethin aint givin one public givin hundreds somethin gotta get sign first middle last wisdom tooth niggas wishin py kissing booth taste breakfast lunch gin juice dinner like dessert french refresh gimme two bite lip come get want lipstick lipgloss hickeys huh doja cat kiss young boy aint got nothin lose oh oh principle baby hold cause like way groove oh oh doja cat boy write name ooh love taste ohlalalalala tongue want boy wri

In [40]:
pipe = Pipeline([('counter_vectorizer', CountVectorizer(max_df=0.8, max_features=200000,
    min_df=0.2, stop_words='english',ngram_range=(1,3))),('tfidf_transform', TfidfTransformer())])

tfidf_matrix = pipe.fit_transform([x for x in df['lyrics']])

In [42]:
similarity_distance = 1 - cosine_similarity(tfidf_matrix)

In [47]:
import numpy as np
def find_similar(song):
    index = df[df["song_id.1"] == song].index[0]
    vector = similarity_distance[int(index),:]
    most_similar = df.iloc[np.argsort(vector)[1], 0]
    return most_similar

print(find_similar(1835514122))

344


In [49]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline

def find_similar_lyrics_by_id(df, song_id, threshold=0.8):
    # Define the pipeline
    pipe = Pipeline([
        ('counter_vectorizer', CountVectorizer(max_df=0.8, max_features=200000, min_df=0.2, stop_words='english', ngram_range=(1,3))),
        ('tfidf_transform', TfidfTransformer())
    ])

    # Fit and transform the lyrics using the pipeline
    tfidf_matrix = pipe.fit_transform([x for x in df['lyrics']])

    # Get the index of the provided song_id
    song_index = df.index[df['song_id.1'] == song_id].tolist()[0]

    # Calculate cosine similarity for the provided song_id
    similarity_scores = list(enumerate(cosine_similarity(tfidf_matrix[song_index], tfidf_matrix)[0]))

    # Filter similar songs based on the threshold
    similar_songs = [(df['song_id.1'][i], score) for i, score in similarity_scores if score > threshold and i != song_index]

    # Print the results
    print(f"Similar lyrics for Song ID {song_id} with a threshold of {threshold}:")
    if not similar_songs:
        print("No similar lyrics found.")
    else:
        for similar_song_id, similarity_score in similar_songs:
            print(f"Similarity Score: {similarity_score:.4f}")
            print(f"Similar Song ID: {similar_song_id}")
            print("=" * 50)

# Call the function with a specific song ID
find_similar_lyrics_by_id(df, song_id=1835514122, threshold=0.8)

Similar lyrics for Song ID 1835514122 with a threshold of 0.8:
Similarity Score: 1.0000
Similar Song ID: 1806516098
Similarity Score: 1.0000
Similar Song ID: 1358244781
Similarity Score: 1.0000
Similar Song ID: 1374086946
Similarity Score: 1.0000
Similar Song ID: 1898528954
Similarity Score: 1.0000
Similar Song ID: 5244822
Similarity Score: 1.0000
Similar Song ID: 442867219
Similarity Score: 1.0000
Similar Song ID: 1862806721
Similarity Score: 1.0000
Similar Song ID: 28496172
Similarity Score: 1.0000
Similar Song ID: 5371229
Similarity Score: 1.0000
Similar Song ID: 516857815
Similarity Score: 0.9544
Similar Song ID: 1487528136
Similarity Score: 0.9446
Similar Song ID: 109218
Similarity Score: 1.0000
Similar Song ID: 1473580610
Similarity Score: 1.0000
Similar Song ID: 1859839343
Similarity Score: 1.0000
Similar Song ID: 434659433
Similarity Score: 1.0000
Similar Song ID: 1362603770
Similarity Score: 1.0000
Similar Song ID: 479219365
Similarity Score: 1.0000
Similar Song ID: 1858481408

# Actual Work Starts Here

In [13]:
import pandas as pd
music_data = pd.read_excel("Lyrics Test.xlsx")

In [14]:
df = music_data.copy()

In [15]:
df['chinese_lyrics'] = df.apply(lambda row: row['tlyrics'] if pd.notnull(row['tlyrics']) else row['lyrics'], axis=1)
df.head()

,song_id,song_id.1,lyrics,tlyrics,chinese_lyrics
0,0,34248839,[00:28.680]ในวันที่โลกนั้นเงียบเหงา แค่มีเธอเด...,[00:28.680]当世界冷冷清清，只有你走过来\n[00:34.970]一切看上去都很萧...,[00:28.680]当世界冷冷清清，只有你走过来\n[00:34.970]一切看上去都很萧...
1,1,462857494,[00:00.00] 作曲 : Keita Haga\n[99:00.00]纯音乐，请欣赏,NaN,[00:00.00] 作曲 : Keita Haga\n[99:00.00]纯音乐，请欣赏
2,2,31654694,[00:13.000]\n[00:19.890]What seems to be endle...,[by:Littamily-G]\n[00:19.890]仿佛一切都是空洞的\n[00:22...,[by:Littamily-G]\n[00:19.890]仿佛一切都是空洞的\n[00:22...
3,3,36471041,[00:00.00] 编曲 : PDP楽\n[01:05.03]好沮丧 the Summer...,NaN,[00:00.00] 编曲 : PDP楽\n[01:05.03]好沮丧 the Summer...
4,4,1417265541,[00:00.00] 作词 : Seiji Nayuta/ナナヲアカリ\n[00:01.00...,[by:我被绑架到]\n[00:20.81]啊~ 搞不懂恋爱的定义是什么啊\n[00:22....,[by:我被绑架到]\n[00:20.81]啊~ 搞不懂恋爱的定义是什么啊\n[00:22....


In [16]:
df.shape

(529, 5)

In [17]:
df = df.dropna(subset='chinese_lyrics')
df.shape

(526, 5)

In [18]:
import re

def remove_timestamps(lyric):
    return re.sub(r'\[\d+:\d+\.\d+\]\s*', '', lyric)

df['lyrics'] = df['lyrics'].apply(remove_timestamps)

In [19]:
df['chinese_lyrics'] = df['chinese_lyrics'].apply(remove_timestamps)

In [20]:
df.head()

,song_id,song_id.1,lyrics,tlyrics,chinese_lyrics
0,0,34248839,ในวันที่โลกนั้นเงียบเหงา แค่มีเธอเดินเข้ามา\nอ...,[00:28.680]当世界冷冷清清，只有你走过来\n[00:34.970]一切看上去都很萧...,当世界冷冷清清，只有你走过来\n一切看上去都很萧条，似乎就要消失不见\n想要做些什么让你知道...
1,1,462857494,作曲 : Keita Haga\n纯音乐，请欣赏,NaN,作曲 : Keita Haga\n纯音乐，请欣赏
2,2,31654694,"What seems to be endless\nPointless, colorless...",[by:Littamily-G]\n[00:19.890]仿佛一切都是空洞的\n[00:22...,[by:Littamily-G]\n仿佛一切都是空洞的\n迷惘的 黯淡的\n你知道的，简直一...
3,3,36471041,编曲 : PDP楽\n好沮丧 the Summer’s Gone 气温又降了 不知不觉又添了...,NaN,编曲 : PDP楽\n好沮丧 the Summer’s Gone 气温又降了 不知不觉又添了...
4,4,1417265541,作词 : Seiji Nayuta/ナナヲアカリ\n作曲 : Seiji Nayuta\nあ...,[by:我被绑架到]\n[00:20.81]啊~ 搞不懂恋爱的定义是什么啊\n[00:22....,[by:我被绑架到]\n啊~ 搞不懂恋爱的定义是什么啊\n首先连喜欢的基准都不知道！\n总之...


# Preprocessing using Jieba and similarity computed using TF-IDF & Cosine Similarity

In [31]:
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

specified_song_id = 1835514122

input_lyrics = df.loc[df['song_id.1'] == specified_song_id, 'chinese_lyrics'].values[0]

def chinese_tokenizer(text):
    return list(jieba.cut(text))

vectorizer = TfidfVectorizer(tokenizer=chinese_tokenizer)
tfidf_matrix = vectorizer.fit_transform(df['chinese_lyrics'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

specified_song_index = df[df['song_id.1'] == specified_song_id].index[0]
similar_songs = cosine_sim[specified_song_index]

most_similar_song_indices = similar_songs.argsort()[:-11:-1]

most_similar_song_ids = df.loc[df.index.isin(most_similar_song_indices), 'song_id.1']

print(f"The top 10 most similar song_id.1 values to song_id.1 = {specified_song_id} are:\n{most_similar_song_ids.tolist()}")

The top 10 most similar song_id.1 values to song_id.1 = 1835514122 are:
[26256023, 1835514122, 3317105, 1831467365, 1753358, 399411296, 28167338, 2013291289, 35447189, 414979758]


In [28]:
len(most_similar_song_indices)

525

# Using BERT

In [32]:
!pip install bert-serving-server
!pip install bert-serving-client

                                              0.0/61.7 kB ? eta -:--:--
     -------------------                      30.7/61.7 kB ? eta -:--:--
     -------------------------              41.0/61.7 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 61.7/61.7 kB 547.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7400 sha256=f5b830465d090c8740ad3fd6e362b431a3d5f1b5fa58976833828bdfbc32f7cc
  Stored in directory: c:\users\fatima azfar\appdata\local\pip\cache\wheels\2b\4d\8f\55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [ ]:
from bert_serving.client import BertClient
import pandas as pd
import jieba
import numpy as np

specified_song_id = 1835514122

input_lyrics = df.loc[df['song_id.1'] == specified_song_id, 'chinese_lyrics'].values[0]

def chinese_tokenizer(text):
    return list(jieba.cut(text))

input_tokens = chinese_tokenizer(input_lyrics)
input_text = ' '.join(input_tokens)

bc = BertClient()

input_embedding = bc.encode([input_text])[0]

similarities = []
for _, row in df.iterrows():
    lyrics_text = ' '.join(chinese_tokenizer(row['chinese_lyrics']))
    lyrics_embedding = bc.encode([lyrics_text])[0]
    similarity = np.dot(input_embedding, lyrics_embedding) / (np.linalg.norm(input_embedding) * np.linalg.norm(lyrics_embedding))
    similarities.append(similarity)

most_similar_song_indices = np.argsort(similarities)[-2:-12:-1]

most_similar_song_ids = df.loc[df.index.isin(most_similar_song_indices), 'song_id.1']

print(f"The top 10 most similar song_id.1 values to song_id.1 = {specified_song_id} are:\n{most_similar_song_ids.tolist()}")

# Bidirectional Temporal Siamese Network

In [22]:
import pandas as pd
import numpy as np
import jieba
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention, Concatenate
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def tokenize_chinese_lyrics(lyrics):
    return list(jieba.cut(lyrics))

df['tokenized_lyrics'] = df['chinese_lyrics'].apply(tokenize_chinese_lyrics)

max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['tokenized_lyrics'])
sequences = tokenizer.texts_to_sequences(df['tokenized_lyrics'])
# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Input layers
input_left = Input(shape=(max_sequence_length,))
input_right = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length)
embedded_left = embedding_layer(input_left)
embedded_right = embedding_layer(input_right)

# Bidirectional LSTM layers
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))
lstm_left = lstm_layer(embedded_left)
lstm_right = lstm_layer(embedded_right)

# Attention mechanism
attention = Attention()([lstm_left, lstm_right])

# Concatenate attention outputs
merged = Concatenate(axis=-1)([lstm_left, lstm_right, attention])

# Dense layers
dense1 = Dense(128, activation='relu')(merged)
dense2 = Dense(64, activation='relu')(dense1)
output = Dense(1, activation='sigmoid')(dense2)

# Model
model = Model(inputs=[input_left, input_right], outputs=output)

# Compilation of the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to compute similarity score between two songs
def compute_similarity(song1_index, song2_index):
    lyrics1 = padded_sequences[song1_index].reshape(1, -1)
    lyrics2 = padded_sequences[song2_index].reshape(1, -1)
    similarity_score = model.predict([lyrics1, lyrics2])[0][0]
    return similarity_score

specified_song_id = 1835514122

specified_song_index = df[df['song_id.1'] == specified_song_id].index[0]

similarity_scores = [compute_similarity(specified_song_index, other_index) for other_index in range(len(df))]

1/1 [==============================] - 1s 541ms/step


1/1 [==============================] - 1s 551ms/step


ValueError: Cannot index with multidimensional key

In [35]:
most_similar_song_indices = sorted(range(len(similarity_scores)), key=lambda k: similarity_scores[k], reverse=True)

top_n = 9

top_similar_songs = df.loc[most_similar_song_indices[:top_n + 1], ['song_id.1', 'chinese_lyrics']]

print(f"The {top_n} most similar songs to song_id.1 = {specified_song_id} are:")
print(top_similar_songs)

The 9 most similar songs to song_id.1 = 1835514122 are:
      song_id.1                                     chinese_lyrics
35    503463966  作词 : 小玖州/鲤小住/千月兔/迟溪/素昭/笙梦/微醉/夜溪/祁言\n作曲 : 李大白/李...
187     3038433                                            纯音乐，请欣赏
31     28819113  [by:Rebecca沫盎]\n让我看看你的身体\n像是马拉卡\n我想带你回家\n跟我来\n...
237  1851258433  作词 : Swei水/DP龙猪/Rays\n作曲 : Swei水/DP龙猪/Rays\n编曲...
272    29343743  如果你想要我的爱\n你得做到他做过的事\n如果你想要这蜜糖Gucci般芳香的双唇\n你最好别...
150    27907218                                            纯音乐，请欣赏
308      108370  作词 : 李宗盛\n作曲 : 李宗盛\n编曲 : Mac Chew\n往事不要再提\n人生已...
519  1356925947                      作词 : 无\n作曲 : SaMZIng\n纯音乐，请欣赏
480  1417481103  已经沉迷于你让我疯狂\n我在倒数321直到你躲好\n你早应该知道你是我的最爱\n绕来绕去我已...
436   436487059  [by:刘玄明童鞋]\n抓紧我的手\n远处的天国\n不是你的归宿\n我心上的每寸肌肉\n都在...


In [32]:
most_similar_song_indices = sorted(range(len(similarity_scores)), key=lambda k: similarity_scores[k], reverse=True)

top_n = 10
top_most_similar_song_indices = most_similar_song_indices[:top_n]

print("Top most similar song indices:", top_most_similar_song_indices)

Top most similar song indices: [35, 187, 31, 237, 272, 150, 308, 519, 480, 436]


# Data without the non-chinese lyrics noise

In [57]:
cleaned_df = music_data.copy()
cleaned_df['chinese_lyrics'] = cleaned_df.apply(lambda row: row['tlyrics'] if pd.notnull(row['tlyrics']) else row['lyrics'], axis=1)
cleaned_df = cleaned_df.dropna(subset='chinese_lyrics')
cleaned_df['chinese_lyrics'] = cleaned_df['chinese_lyrics'].apply(remove_timestamps)
cleaned_df.head()

,song_id,song_id.1,lyrics,tlyrics,chinese_lyrics
0,0,34248839,[00:28.680]ในวันที่โลกนั้นเงียบเหงา แค่มีเธอเด...,[00:28.680]当世界冷冷清清，只有你走过来\n[00:34.970]一切看上去都很萧...,当世界冷冷清清，只有你走过来\n一切看上去都很萧条，似乎就要消失不见\n想要做些什么让你知道...
1,1,462857494,[00:00.00] 作曲 : Keita Haga\n[99:00.00]纯音乐，请欣赏,NaN,作曲 : Keita Haga\n纯音乐，请欣赏
2,2,31654694,[00:13.000]\n[00:19.890]What seems to be endle...,[by:Littamily-G]\n[00:19.890]仿佛一切都是空洞的\n[00:22...,[by:Littamily-G]\n仿佛一切都是空洞的\n迷惘的 黯淡的\n你知道的，简直一...
3,3,36471041,[00:00.00] 编曲 : PDP楽\n[01:05.03]好沮丧 the Summer...,NaN,编曲 : PDP楽\n好沮丧 the Summer’s Gone 气温又降了 不知不觉又添了...
4,4,1417265541,[00:00.00] 作词 : Seiji Nayuta/ナナヲアカリ\n[00:01.00...,[by:我被绑架到]\n[00:20.81]啊~ 搞不懂恋爱的定义是什么啊\n[00:22....,[by:我被绑架到]\n啊~ 搞不懂恋爱的定义是什么啊\n首先连喜欢的基准都不知道！\n总之...


In [58]:
cleaned_df = cleaned_df[~cleaned_df['chinese_lyrics'].str.contains("纯音乐，请欣赏")]
cleaned_df = cleaned_df[~cleaned_df['chinese_lyrics'].str.contains("N/A")]

In [60]:
import pandas as pd
import numpy as np
import jieba
from keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Attention, Concatenate
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def tokenize_chinese_lyrics(lyrics):
    return list(jieba.cut(lyrics))

cleaned_df['tokenized_lyrics'] = cleaned_df['chinese_lyrics'].apply(tokenize_chinese_lyrics)

max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(cleaned_df['tokenized_lyrics'])
sequences = tokenizer.texts_to_sequences(cleaned_df['tokenized_lyrics'])

# Pad sequences to have consistent length
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Input layers
input_left = Input(shape=(max_sequence_length,))
input_right = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length)
embedded_left = embedding_layer(input_left)
embedded_right = embedding_layer(input_right)

# Bidirectional LSTM layers
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))
lstm_left = lstm_layer(embedded_left)
lstm_right = lstm_layer(embedded_right)

# Attention mechanism
attention = Attention()([lstm_left, lstm_right])

# Concatenate attention outputs
merged = Concatenate(axis=-1)([lstm_left, lstm_right, attention])

# Dense layers
dense1 = Dense(128, activation='relu')(merged)
dense2 = Dense(64, activation='relu')(dense1)
output = Dense(1, activation='sigmoid')(dense2)

# Model
model = Model(inputs=[input_left, input_right], outputs=output)

# Compilation of the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to compute similarity score between two songs
def compute_similarity(song1_index, song2_index):
    lyrics1 = padded_sequences[song1_index].reshape(1, -1)
    lyrics2 = padded_sequences[song2_index].reshape(1, -1)
    similarity_score = model.predict([lyrics1, lyrics2])[0][0]
    return similarity_score

specified_song_id = 1835514122

specified_song_index = cleaned_df[cleaned_df['song_id.1'] == specified_song_id].index[0]

similarity_scores = [compute_similarity(specified_song_index, other_index) for other_index in range(len(cleaned_df))]

1/1 [==============================] - 0s 500ms/step


1/1 [==============================] - 1s 512ms/step


In [61]:
most_similar_song_indices = sorted(range(len(similarity_scores)), key=lambda k: similarity_scores[k], reverse=True)

top_n = 9

top_similar_songs = df.loc[most_similar_song_indices[:top_n + 1], ['song_id.1', 'chinese_lyrics']]

print(f"The {top_n} most similar songs to song_id.1 = {specified_song_id} are:")
print(top_similar_songs)

The 9 most similar songs to song_id.1 = 1835514122 are:
      song_id.1                                     chinese_lyrics
325      179765  作词 : 刘航\n作曲 : 刘航\n看着你慢慢离开我的视线\n才明白原来这份爱情已走远\n而...
354   425295473  作词 : TT\n编曲：Blasina\n混音：Mai\n我在菩萨面前许下心愿\n希望爸爸妈...
32     28768887  [by:SKWLHONG]\n[ti:]\n[ar:]\n[al:]\n[by:九九Lrc歌...
170   484249508  [by:妈的傻逼腾讯]\n“我的出生是某人的期望”\n我这么坚信着，一直活着\n无论做什么都...
137      191084  作词 : 谢娜\n作曲 : 谢娜\n编曲 : 常石磊\n小时候总爱抬头看白云朵朵\n哼着我最...
338   399411296  [Refrain]\nEverybody getting high\nGetting hig...
265    21301195  [by:每天只收一颗稻谷]\n微风拂过喃喃低语的神秘树丛，带我回家\n仿佛诉说着希望的故事，...
270    31421243  [by:Utadal]\n如果我有把今晚的咖哩\n一滴不剩的吃光是不是比较好呢\n印象中你深...
83   1378869403  作词 : 张志远\n作曲 : 思默\n加油鸭 加油鸭 向前冲鸭\n苍天鸭 大地鸭 我的马鸭\...
283     1753358  [by:青空味的橙子]\n你一定还记得\n接吻就是最纯粹的接吻\n叹息就是最纯粹的叹息\n单...
